# Magic methods ("dunders")

## Operators

In [1]:
class Gymatria:

    _aleph_beth = None
    
    def __init__(self, expression:str) -> None:
        self._expr = expression
        self._expr_value = Gymatria.get_value(expression)
    
    @property
    def expr(self) -> str:
        return self._expr

    @property
    def expr_value(self)->int:
        return self._expr_value

    def __add__(self , other) -> int:  # operator+
        return self.expr_value + other.expr_value

    def __sub__(self , other) -> int:  # operator-
        return abs(self.expr_value - other.expr_value)
    
    def __mul__(self , other) -> int:  # operator*
        return self.expr_value * other.expr_value
    
    def __str__(self) -> str:
        return f"{self.expr} בגימטריה זה: {self.expr_value}"


    @classmethod
    def get_value(cls,expression:str) -> int:
        aleph_beth = cls.get_aleph_beth()
        expr_value = 0
        for ot in expression:
            if ord('א') <= ord(ot) <= ord('ת'):
                expr_value += aleph_beth[ot]
        return expr_value

    @classmethod
    def get_aleph_beth(cls):
        if cls._aleph_beth == None:
            cls._set_aleph_beth()
        return cls.aleph_beth

    @classmethod
    def ot_sofit(cls, ot: str)-> bool:
        cls.otiot_sofiot = ['ץ','ך','ף','ן','ם']
        if ot in cls.otiot_sofiot:
            return True
        return False

    @classmethod
    def _set_aleph_beth(cls) -> None:
        ot_num = ord('א')
        cls.aleph_beth={}
        val = 1
        for i in range(27):
            cls.aleph_beth[chr(ot_num+i)] = val
            if not cls.ot_sofit(chr(ot_num+i)):
                if 90 >= val >= 10:
                    val+=10
                elif val >= 100:
                    val+= 100 
                else: val+=1

In [2]:

aba = Gymatria('אבא')
ima = Gymatria('אמא')

print(aba)  #  calls __str__ method
print(ima)
print(f'אבא+אמא בגימטריה = {ima+aba}')  # calls __add__
print(f'אבא-אמא בגימטריה = {aba-ima}')
print(f'אבא*אמא בגימטריה = {ima*aba}')

אבא בגימטריה זה: 4
אמא בגימטריה זה: 42
אבא+אמא בגימטריה = 46
אבא-אמא בגימטריה = 38
אבא*אמא בגימטריה = 168


## Construction: the 'new' dunder

In [3]:
# Example based on sympy
from abc import abstractmethod

class Expression:
    @abstractmethod
    def evaluate(self, variable_value):
        pass

class Variable(Expression):
    def evaluate(self, variable_value):
        return variable_value

class Addition(Expression):
    def __init__(self, lhs, rhs):
        self.lhs = lhs
        self.rhs = rhs
    def evaluate(self, variable_value):
        return self.lhs.evaluate(variable_value) + self.rhs.evaluate(variable_value)

class Subtraction(Expression):
    def __init__(self, lhs, rhs):
        self.lhs = lhs
        self.rhs = rhs
    def evaluate(self, variable_value):
        return self.lhs.evaluate(variable_value) - self.rhs.evaluate(variable_value)


v = Variable()

e1 = Addition(v,v)
print(type(e1))
print(e1.evaluate(5))

e2 = Subtraction(e1,v)
print(type(e2))
print(e2.evaluate(6))

<class '__main__.Addition'>
10
<class '__main__.Subtraction'>
6


In [24]:
# Example based on sympy
from abc import abstractmethod


class Expression:
    """
    Defines an expression in a single variable.
    """
    def __new__(cls, *args, **kwargs):
        if len(args)>=3:
            lhs, operator, rhs = args
            if operator=='+':
                cls=Addition
            elif operator=='-':
                cls=Subtraction
            else:
                raise ValueError(f"Unknown operator {operator}")
        return super().__new__(cls)

    @abstractmethod
    def evaluate(self, variable_value):
        pass

class Variable(Expression):
    def evaluate(self, variable_value):
        return variable_value

class Addition(Expression):
    def __init__(self, lhs, _, rhs):
        self.lhs = lhs
        self.rhs = rhs

    def evaluate(self, variable_value):
        return self.lhs.evaluate(variable_value) + self.rhs.evaluate(variable_value)

class Subtraction(Expression):
    def __init__(self, lhs, _, rhs):
        self.lhs = lhs
        self.rhs = rhs

    def evaluate(self, variable_value):
        return self.lhs.evaluate(variable_value) - self.rhs.evaluate(variable_value)


v = Variable()

e1 = Expression(v, '+', v)
print(type(e1))
print(e1.evaluate(5))

e2 = Expression(e1, '-', v)
print(type(e2))
print(e2.evaluate(6))

<class '__main__.Addition'>
10
<class '__main__.Subtraction'>
6


## Adding and deleting fields and methods

In [4]:
# It is possible to add new fields "on the fly":

class Empty:
    pass
empty = Empty()
empty.something = "something"
print(empty.something)

something


In [5]:
# It is possible to add new methods "on the fly":
def iadd(self, other):
    self._expr += other._expr
    self._expr_value += other._expr_value
    return self
Gymatria.__iadd__ = iadd   # operator+=

aba += ima
print(aba)

אבאאמא בגימטריה זה: 46


In [6]:
# Deleting variables:

x = [1,2]
print(x)
del x
print(x) #=> NameError

[1, 2]


NameError: name 'x' is not defined

In [7]:
# Deleting methods:

print(aba)
del Gymatria.__str__
print(aba)

אבאאמא בגימטריה זה: 46


## Casting

In [29]:
Gymatria.__int__ = lambda self: self.expr_value
Gymatria.__str__ = lambda self: f"{self.expr} בגימטריה זה: {self.expr_value}"  # __str__ is for end-users.
Gymatria.__float__ = lambda self: float(self.expr_value)

In [30]:
print(int(aba))
print(str(aba))
print(float(aba))

46
אבאאמא בגימטריה זה: 46
46.0


In [9]:
Gymatria.__repr__ = lambda self: f"Gymatria({self.expr!r})"  # __repr__ is for unique system representation.
shalom = Gymatria("שלום")
print(shalom)
print(repr(shalom))
shalom2 = eval(repr(shalom))
print(shalom2)
gymatria_list = [shalom,Gymatria("עולם")]
print("List of reprs: ", gymatria_list)
print("List of strs : ", [str(x) for x in gymatria_list])
print("List of strs : ", list(map(str, gymatria_list)))

Gymatria('שלום')
Gymatria('שלום')
Gymatria('שלום')
List of reprs:  [Gymatria('שלום'), Gymatria('עולם')]
List of strs :  ["Gymatria('שלום')", "Gymatria('עולם')"]
List of strs :  ["Gymatria('שלום')", "Gymatria('עולם')"]


In [10]:
gymatria_list.append(Gymatria('צה"ל'))
gymatria_list.append(Gymatria("וכו'"))
print(gymatria_list)

[Gymatria('שלום'), Gymatria('עולם'), Gymatria('צה"ל'), Gymatria("וכו'")]


## Context manager - enter and exit

In [11]:
class ContextManager(): 
    def __init__(self): 
        print('init method called') 
    def __enter__(self): 
        print('enter method called') 
        return self
    def __exit__(self, exc_type, exc_value, exc_traceback): 
        print('exit method called') 

with ContextManager() as manager:
    print('with statement block') 

init method called
enter method called
with statement block
exit method called


In [12]:
# This example illustrates how "with" works with files:

class FileManager():
    def __init__(self, filename, mode, encoding="utf-8"): 
        self.filename = filename 
        self.mode = mode 
        self.file = None
        self.encoding = encoding

    def __enter__(self):  
        self.file = open(self.filename, self.mode ,encoding=self.encoding)
        return self.file
      
    def __exit__(self, exc_type, exc_value, exc_traceback): 
        self.file.close() 
  
# Using file manager:
with FileManager('test.txt', 'w') as f: 
    for word in ["a","b","c"]:
        f.write(str(word)+'\n')
    print("In with: ", f.closed)        
print("After with: ", f.closed)

# Using Python's file:
with open('test.txt', 'w') as f:
    for word in ["a","b","c"]:
        f.write(str(word)+'\n')
    print("In with: ", f.closed)
print("After with: ", f.closed)

In with:  False
After with:  True
In with:  False
After with:  True


## Operators on lists and sets

In [13]:
class Node:
    def __init__(self,data:object = None, next = None ,prev = None)->None:
        self.data = data
        self.next = next
        self.prev = prev
    def __repr__(self) ->str:
        return f"Node({self.data!r})"

class LinkedList:
    def __init__(self) ->None:
        self._head = None
        self._tail = Node(None)
        self._size = 0

    def __len__(self) ->int:  # operator len(...)
        return self._size

    def _getnode(self, index:int):  
        if index >= self._size or index < 0: raise IndexError("invalid input, index out of range")
        temp = self._head
        for i in range(index):
            temp = temp.next
        return temp
    
    def __getitem__(self, index:int):  # operator[] for reading
        return self._getnode(index).data

    def __setitem__(self , index:int, value:object) ->None:   # operator[] for writing
        self._getnode(index).data = value
    
    def __contains__(self , value:object) ->None:   # operator "in"
        temp = self._head
        while temp is not None:
            if temp.data==value:
                return True
            temp = temp.next
        return False
    
    def __delitem__(self , index:int) ->None:   # operator "del"
        if index==0:
            self._head = self._getnode(index+1)
            self._head.prev = None
        else:
            prev_node = self._getnode(index-1)
            next_node = self._getnode(index+1)
            prev_node.next , next_node.prev = next_node ,prev_node
        self._size-=1
    
    def insert(self, new_value , index :int = None):
        new_node = Node(new_value)
        if index == None:
            if self._head == None:
                self._head = new_node
                self._head.next = self._tail
                self._tail.prev = self._head
                self._head.prev = None
            else:
                (self._tail.prev).next, new_node.prev = new_node , self._tail.prev
                self._tail.prev , new_node.next =  new_node , self._tail
            self._size += 1
            return self

    def __str__(self) -> str:
        s = " "
        temp = self._head
        while temp != self._tail:
            if temp != self._tail.prev:
               s += f"{temp} -> "
            else: s += f"{temp}"
            temp = temp.next
        return s

    def __repr__(self) ->str:
        temp = self._head
        s = "LinkedList()"
        while temp != self._tail:
            s += f".insert({temp.data})"

            temp = temp.next
        return s

In [14]:
linked_list = LinkedList()
# linked_list[0] = 3  => IndexError
# linked_list[len(linked_list)] => IndexError
for i in range(1,15,2):
    linked_list.insert(i)
print(f"linked_list = {linked_list}")  # calls __str__ method
print(f"linked_list[3] = {linked_list[3]}")   # calls __getitem__ method
print(f"len(linked_list) = {len(linked_list)}")   # calls __len__ method
print(f"3 in linked_list: {3 in linked_list}. 4 in linked_list: {4 in linked_list}. ")
r = repr(linked_list)   # calls __repr__ method
print(r)
linl = eval(r)   # calls __repr__ method
linl[3] = "Tom Pythonovitz"  # calls __setitem__ method
print(linl)
del(linl[0])   # calls __delitem__ method
print(linl)

linked_list =  Node(1) -> Node(3) -> Node(5) -> Node(7) -> Node(9) -> Node(11) -> Node(13)
linked_list[3] = 7
len(linked_list) = 7
3 in linked_list: True. 4 in linked_list: False. 
LinkedList().insert(1).insert(3).insert(5).insert(7).insert(9).insert(11).insert(13)
 Node(1) -> Node(3) -> Node(5) -> Node('Tom Pythonovitz') -> Node(9) -> Node(11) -> Node(13)
 Node(3) -> Node(5) -> Node('Tom Pythonovitz') -> Node(9) -> Node(11) -> Node(13)
